## Agosto

In [25]:
val ag95 = spark.read.text("C:/Users/nora.hafidi/Desktop/Big Data/NASA/NASA_access_log_Aug95/access_log_Aug95")

ag95: org.apache.spark.sql.DataFrame = [value: string]


In [2]:
ag95.show(3, false)

+--------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                     |
+--------------------------------------------------------------------------------------------------------------------------+
|in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839|
|uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0                                                   |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0                          |
+--------------------------------------------------------------------------------------------------------------------------+
only showing top 3 rows



In [26]:
val expresion = """(.*)[\s-]{4}\s\[(\d\d)\/.*\/1995:(\d\d):(\d\d):(\d\d).*\s\"\s?([A-Z]*)\s(\/\S*).*?(HTTP.*)?\"\s?(\d\d\d)\s(\d*)"""

expresion: String = (.*)[\s-]{4}\s\[(\d\d)\/.*\/1995:(\d\d):(\d\d):(\d\d).*\s\"\s?([A-Z]*)\s(\/\S*).*?(HTTP.*)?\"\s?(\d\d\d)\s(\d*)


In [27]:
val ag95_prima = ag95
    .select(regexp_extract($"value",expresion,1).as("host"),
            regexp_extract($"value",expresion,2).as("day"),
            regexp_extract($"value",expresion,3).as("hours"),
            regexp_extract($"value",expresion,4).as("minutes"),
            regexp_extract($"value",expresion,5).as("seconds"),
            regexp_extract($"value",expresion,6).as("request"),
            regexp_extract($"value",expresion,7).as("resource"),
            regexp_extract($"value",expresion,8).as("protocol"),
            regexp_extract($"value",expresion,9).cast("int").as("http_status_code"),
            regexp_extract($"value",expresion,10).cast("int").as("size"))

ag95_prima: org.apache.spark.sql.DataFrame = [host: string, day: string ... 8 more fields]


In [28]:
ag95_prima.show(3, false)

+-----------------+---+-----+-------+-------+-------+-----------------------------------------------+--------+----------------+----+
|host             |day|hours|minutes|seconds|request|resource                                       |protocol|http_status_code|size|
+-----------------+---+-----+-------+-------+-------+-----------------------------------------------+--------+----------------+----+
|in24.inetnebr.com|01 |00   |00     |01     |GET    |/shuttle/missions/sts-68/news/sts-68-mcc-05.txt|HTTP/1.0|200             |1839|
|uplherc.upl.com  |01 |00   |00     |07     |GET    |/                                              |HTTP/1.0|304             |0   |
|uplherc.upl.com  |01 |00   |00     |08     |GET    |/images/ksclogo-medium.gif                     |HTTP/1.0|304             |0   |
+-----------------+---+-----+-------+-------+-------+-----------------------------------------------+--------+----------------+----+
only showing top 3 rows



In [7]:
val nulls_h = ag95_prima
.select("http_status_code")
.where($"http_status_code" .isNull)
.count()

nulls: Long = 67


In [18]:
val cons1 = ag95_prima
.select("protocol")
.where($"protocol" !== "")
.groupBy("protocol")
.count()
cons1.show(false)

+-------------------------------------------------+-------+
|protocol                                         |count  |
+-------------------------------------------------+-------+
|HTTP/1.0From:  <berend@blazemonger.pc.cc.cmu.edu>|624    |
|HTTP/1.0                                         |113    |
|HTTP/V1.0                                        |163    |
|HTTP/1.0                                         |1567056|
+-------------------------------------------------+-------+



cons1: org.apache.spark.sql.DataFrame = [protocol: string, count: bigint]


In [96]:
val cons2 = ag95_prima
.select("http_status_code")
.groupBy("http_status_code")
.count()
.orderBy(desc("count"))
cons2.show()

+----------------+-------+
|http_status_code|  count|
+----------------+-------+
|             200|1398988|
|             304| 134146|
|             302|  26444|
|             404|  10052|
|             403|    171|
|            null|     67|
|             501|     27|
|             500|      3|
+----------------+-------+



cons2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [http_status_code: int, count: bigint]


In [95]:
val cons3 = ag95_prima
.select("request")
.groupBy("request")
.count()
.orderBy(desc("count"))
cons3.show()

+-------+-------+
|request|  count|
+-------+-------+
|    GET|1565755|
|   HEAD|   3965|
|   POST|    111|
|       |     67|
+-------+-------+



cons3: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [request: string, count: bigint]


In [94]:
val cons4 = ag95_prima
.groupBy("resource")
.count()
.orderBy(desc("count"))
cons4.show(1, false)

+--------------------------+-----+
|resource                  |count|
+--------------------------+-----+
|/images/NASA-logosmall.gif|97410|
+--------------------------+-----+
only showing top 1 row



cons4: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [resource: string, count: bigint]


In [93]:
val cons5 = ag95_prima
.orderBy(desc("size"))
cons5.show(1, false)

+--------------+---+-----+-------+-------+-------+-----------------------------------------------+----------------+-------+
|host          |day|hours|minutes|seconds|request|resource                                       |http_status_code|size   |
+--------------+---+-----+-------+-------+-------+-----------------------------------------------+----------------+-------+
|163.205.156.16|03 |15   |51     |23     |GET    |/statistics/1995/Jul/Jul95_reverse_domains.html|200             |3421948|
+--------------+---+-----+-------+-------+-------+-----------------------------------------------+----------------+-------+
only showing top 1 row



cons5: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [host: string, day: string ... 7 more fields]


In [92]:
val cons6 = ag95_prima
.select("day")
.groupBy("day")
.count()
.orderBy(desc("count"))
cons6.show(3, false)

+---+-----+
|day|count|
+---+-----+
|31 |90121|
|30 |80640|
|29 |67988|
+---+-----+
only showing top 3 rows



cons6: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [day: string, count: bigint]


In [91]:
val cons7 = ag95_prima
.select("host")
.groupBy("host")
.count()
.orderBy(desc("count"))
cons7.show(5, false)

+--------------------+-----+
|host                |count|
+--------------------+-----+
|edams.ksc.nasa.gov  |6530 |
|piweba4y.prodigy.com|4846 |
|163.206.89.4        |4791 |
|piweba5y.prodigy.com|4607 |
|piweba3y.prodigy.com|4416 |
+--------------------+-----+
only showing top 5 rows



cons7: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [host: string, count: bigint]


In [90]:
val cons8 = ag95_prima
.select("hours")
.groupBy("hours")
.count()
.orderBy(desc("count"))
cons8.show(5, false)

+-----+------+
|hours|count |
+-----+------+
|15   |109461|
|12   |105143|
|13   |104535|
|14   |101392|
|16   |99527 |
+-----+------+
only showing top 5 rows



cons8: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [hours: string, count: bigint]


In [97]:
val cons9 = ag95_prima
.select("day")
.where($"http_status_code" === "404")
.groupBy("day")
.count()
.orderBy("day")
cons9.show(31)

+---+-----+
|day|count|
+---+-----+
| 01|  243|
| 03|  304|
| 04|  344|
| 05|  236|
| 06|  373|
| 07|  537|
| 08|  391|
| 09|  279|
| 10|  315|
| 11|  263|
| 12|  196|
| 13|  216|
| 14|  287|
| 15|  327|
| 16|  259|
| 17|  270|
| 18|  256|
| 19|  208|
| 20|  312|
| 21|  305|
| 22|  288|
| 23|  345|
| 24|  420|
| 25|  415|
| 26|  366|
| 27|  370|
| 28|  410|
| 29|  420|
| 30|  571|
| 31|  526|
+---+-----+



cons9: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [day: string, count: bigint]


## Julio

In [19]:
val jul95 = spark.read.text("C:/Users/nora.hafidi/Desktop/Big Data/NASA/NASA_access_log_Jul95/access_log_Jul95")

jul95: org.apache.spark.sql.DataFrame = [value: string]


In [20]:
jul95.show(3, false)

+--------------------------------------------------------------------------------------------------------------------+
|value                                                                                                               |
+--------------------------------------------------------------------------------------------------------------------+
|199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245                              |
|unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985                   |
|199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085|
+--------------------------------------------------------------------------------------------------------------------+
only showing top 3 rows



In [22]:
val jul95_prima = jul95
    .select(regexp_extract($"value",expresion,1).as("host"),
            regexp_extract($"value",expresion,2).as("day"),
            regexp_extract($"value",expresion,3).as("hours"),
            regexp_extract($"value",expresion,4).as("minutes"),
            regexp_extract($"value",expresion,5).as("seconds"),
            regexp_extract($"value",expresion,6).as("request"),
            regexp_extract($"value",expresion,7).as("resource"),
            regexp_extract($"value",expresion,8).as("protocol"),
            regexp_extract($"value",expresion,9).cast("int").as("http_status_code"),
            regexp_extract($"value",expresion,10).cast("int").as("size"))

jul95_prima: org.apache.spark.sql.DataFrame = [host: string, day: string ... 8 more fields]


In [24]:
jul95_prima.show(3, false)

+--------------------+---+-----+-------+-------+-------+--------------------------------------------+--------+----------------+----+
|host                |day|hours|minutes|seconds|request|resource                                    |protocol|http_status_code|size|
+--------------------+---+-----+-------+-------+-------+--------------------------------------------+--------+----------------+----+
|199.72.81.55        |01 |00   |00     |01     |GET    |/history/apollo/                            |HTTP/1.0|200             |6245|
|unicomp6.unicomp.net|01 |00   |00     |06     |GET    |/shuttle/countdown/                         |HTTP/1.0|200             |3985|
|199.120.110.21      |01 |00   |00     |09     |GET    |/shuttle/missions/sts-73/mission-sts-73.html|HTTP/1.0|200             |4085|
+--------------------+---+-----+-------+-------+-------+--------------------------------------------+--------+----------------+----+
only showing top 3 rows



In [29]:
val nulls_h_j = jul95_prima
.select("http_status_code")
.where($"http_status_code" .isNull)
.count()

nulls_h_j: Long = 13


In [30]:
val cons1_j = jul95_prima
.select("protocol")
.where($"protocol" !== "")
.groupBy("protocol")
.count()
cons1_j.show(false)

+-------------------------------------------------+-------+
|protocol                                         |count  |
+-------------------------------------------------+-------+
|HTTP/1.0From:  <berend@blazemonger.pc.cc.cmu.edu>|611    |
|HTTP/*                                           |13     |
|HTTP/1.0                                         |15     |
|HTTP/V1.0                                        |116    |
|HTTP/1.0                                         |1887956|
+-------------------------------------------------+-------+



cons1_j: org.apache.spark.sql.DataFrame = [protocol: string, count: bigint]


In [31]:
val cons2_j = jul95_prima
.select("http_status_code")
.groupBy("http_status_code")
.count()
.orderBy(desc("count"))
cons2_j.show()

+----------------+-------+
|http_status_code|  count|
+----------------+-------+
|             200|1701534|
|             304| 132627|
|             302|  46571|
|             404|  10840|
|             500|     62|
|             403|     54|
|             501|     14|
|            null|     13|
+----------------+-------+



cons2_j: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [http_status_code: int, count: bigint]


In [32]:
val cons3_j = jul95_prima
.select("request")
.groupBy("request")
.count()
.orderBy(desc("count"))
cons3_j.show()

+-------+-------+
|request|  count|
+-------+-------+
|    GET|1887639|
|   HEAD|   3952|
|   POST|    111|
|       |     13|
+-------+-------+



cons3_j: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [request: string, count: bigint]


In [33]:
val cons4_j = jul95_prima
.groupBy("resource")
.count()
.orderBy(desc("count"))
cons4_j.show(1, false)

+--------------------------+------+
|resource                  |count |
+--------------------------+------+
|/images/NASA-logosmall.gif|111388|
+--------------------------+------+
only showing top 1 row



cons4_j: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [resource: string, count: bigint]


In [34]:
val cons5_j = jul95_prima
.orderBy(desc("size"))
cons5_j.show(1, false)

+-----+---+-----+-------+-------+-------+---------------------------------------+--------+----------------+-------+
|host |day|hours|minutes|seconds|request|resource                               |protocol|http_status_code|size   |
+-----+---+-----+-------+-------+-------+---------------------------------------+--------+----------------+-------+
|derec|07 |14   |03     |32     |GET    |/shuttle/countdown/video/livevideo.jpeg|HTTP/1.0|200             |6823936|
+-----+---+-----+-------+-------+-------+---------------------------------------+--------+----------------+-------+
only showing top 1 row



cons5_j: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [host: string, day: string ... 8 more fields]


In [36]:
val cons6_j = jul95_prima
.select("day")
.groupBy("day")
.count()
.orderBy(desc("count"))
cons6_j.show(3, false)

+---+------+
|day|count |
+---+------+
|13 |134202|
|06 |100960|
|05 |94575 |
+---+------+
only showing top 3 rows



cons6_j: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [day: string, count: bigint]


In [37]:
val cons7_j = jul95_prima
.select("host")
.groupBy("host")
.count()
.orderBy(desc("count"))
cons7_j.show(5, false)

+--------------------+-----+
|host                |count|
+--------------------+-----+
|piweba3y.prodigy.com|17572|
|piweba4y.prodigy.com|11591|
|piweba1y.prodigy.com|9868 |
|alyssa.prodigy.com  |7852 |
|siltb10.orl.mmc.com |7573 |
+--------------------+-----+
only showing top 5 rows



cons7_j: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [host: string, count: bigint]


In [38]:
val cons8_j = jul95_prima
.select("hours")
.groupBy("hours")
.count()
.orderBy(desc("count"))
cons8_j.show(5, false)

+-----+------+
|hours|count |
+-----+------+
|14   |122477|
|12   |122085|
|15   |121199|
|13   |120813|
|16   |118036|
+-----+------+
only showing top 5 rows



cons8_j: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [hours: string, count: bigint]


In [39]:
val cons9_j = jul95_prima
.select("day")
.where($"http_status_code" === "404")
.groupBy("day")
.count()
.orderBy("day")
cons9_j.show(31)

+---+-----+
|day|count|
+---+-----+
| 01|  315|
| 02|  290|
| 03|  474|
| 04|  359|
| 05|  497|
| 06|  640|
| 07|  570|
| 08|  302|
| 09|  348|
| 10|  398|
| 11|  471|
| 12|  470|
| 13|  531|
| 14|  413|
| 15|  254|
| 16|  257|
| 17|  406|
| 18|  465|
| 19|  639|
| 20|  428|
| 21|  334|
| 22|  192|
| 23|  233|
| 24|  328|
| 25|  461|
| 26|  335|
| 27|  336|
| 28|   94|
+---+-----+



cons9_j: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [day: string, count: bigint]
